In [1]:
import numpy as np
import pandas as pd
import re, sys, os, csv
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

In [2]:
#stop_words = list(get_stop_words('en'))         #About 900 stop words
#nltk_words = list(stopwords.words('english'))   #About 150 stop words
#stop_words.extend(nltk_words)

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('dataset/wordlists/merged.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 0
    elif label == "sadness":return 3
    elif label == "enthusiasm":return 1
    elif label == "neutral":return 0
    elif label == "worry":return 3
    elif label == "surprise":return 2
    elif label == "love":return 2
    elif label == "fun":return 1
    elif label == "hate":return 4
    elif label == "happiness":return 1
    elif label == "boredom":return 0
    elif label == "relief":return 1
    elif label == "anger":return 5

In [3]:
def read_csv(file, lst):
    with open(file, newline='') as f:
        reader = csv.reader(f)
        for i,row in enumerate(reader):
            if i > 0:
                tweet = clean_tweet(str(row[1]))
                lst.append(tweet)

In [7]:
hate = []
read_csv("hate.csv", hate)
"""worry = []
read_csv("raw_data/worried.csv", worry)
#read_csv("worry.csv", worry)
read_csv("raw_data/anxiety.csv", worry)
happy = []
read_csv("raw_data/happy.csv", happy)
sad = []
read_csv("raw_data/sad.csv", sad)"""

dataWriter = csv.writer(open('hate_processes.csv', 'w'), delimiter=',',lineterminator="\n")
for tweet in hate:
    dataWriter.writerow([tweet, 8])

In [9]:
count = 0
with open('data_new.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1
print(count)

5309
